In [1]:
import numpy as np
import random
import csv

data = []

# Betöltés
# kódolási séma specifikálása
with open('data.csv','r', encoding='utf-8-sig') as f:
    csvreader = csv.DictReader(f)
    for item in csvreader:
        data.append([ item['DATE'], item['AUTHOR'], item['CONTENT'], item['CLASS'] ])

# A 'data' tömb elemei: ['dátum string', 'szerző', 'komment', 'osztály cimke ('0': nem spam, '1': spam)']
        
# Train/test szétválasztás
split = 0.7
data = np.asarray(data)
perm = np.random.permutation(len(data))

train = data[perm][0:int(len(data)*split)]
test = data[perm][int(len(data)*split):]

print('Train set: ', np.shape(train))
print('Test set: ', np.shape(test))

# Buta osztályozó
def dumb_classify(data):
    threshold = 0.3
    if random.random() > threshold:
        return '1'
    else:
        return '0'


# Használd a 'train' adatokat az osztályozó módszer kidolgozására, a 'test' adatokat kiértékelésére!
# Lehetőleg használj gépi tanulást!
# Dokumentáld az érdekesnek tartott kísérleteket is!

# Példa kiértékelés 'recall' számításával. 
# Kérdés: Milyen egyéb metrikát használnál kiértékelésre és miért? 
sum_positive = 0
found_positive = 0

for datapoint in test:
    if datapoint[-1] == '1':
        sum_positive += 1
        if dumb_classify(datapoint) == '1':
            found_positive += 1
    
print('Recall:', found_positive / sum_positive)

Train set:  (1367, 4)
Test set:  (586, 4)
Recall: 0.717607973421927


## Importálandó modulok

In [2]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import emoji
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

tokenizer = TweetTokenizer()
stemmer = EnglishStemmer()
stp_wrds = set(stopwords.words('english') + [''])

## Előfeldolgozás

In [3]:
train_df = pd.DataFrame(train, columns=["date", "author", "text", "spam"])
test_df = pd.DataFrame(test, columns=["date", "author", "text", "spam"])

In [4]:
def process_word(word):
    # szamok helyettesitese
    if re.match(r'\d+', word) is not None:
        return '$'
    # emoji es smiley helyettesítése
    elif re.match(r'(?::|;|=)(?:-)?(?:\)|D|P|\()', word) is not None or word in emoji.UNICODE_EMOJI:
        return 'EMOJI'
    # url linkek helyettesítése
    elif re.match(r'((http|https):\/\/)?([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])?', word) is not None:
        return 'URL'
    else:
        # írásjelek eltávolítása a felkiáltójelek kivételével
        out = re.sub(r'[^A-Za-z0-9!]', '', word)
        # ha egy betű egymás utan több, mint háromszor szerepel egy szóban, akkor valószínűleg emfatikus a használata
        # pl. woooowwwwwwww
        # ezeket egyetlen betűre cserélem
        out = re.sub(r'(.)\1+', r'\1', out)
        # stemmelés
        return stemmer.stem(out)
    
def preprocessor(comment_text):
    # \ufeff karakterek eltűntetése
    clean_text = comment_text.replace('\ufeff', ' ')
    # html kódok eltűntetése
    clean_text = re.sub(r'<br\s*\/*>', ' ', clean_text)
    # kisbetűsítés és tokenizálás
    token_list = tokenizer.tokenize(clean_text.lower())
    # előfeldolgozás, címkékre cserélés, stemmelés
    p_token_list = [process_word(word) for word in token_list]
    # stopword szavak eltávolítása
    return [item for item in p_token_list if item not in stp_wrds]

In [5]:
x = train_df['text'][3]
print('Egy példa a preprocesszor működéséről')
print('Eredeti komment:')
print(x)
print('Előfeldolgozás utáni komment:')
print(preprocessor(x))
#preprocessor('HAHAA THIS DANCE IS TIGHTTTT<br /><br />I know :) !!!"^*&^?')

Egy példa a preprocesszor működéséről
Eredeti komment:
Check out my Music Videos! Fuego - U LA LA Remix  hyperurl.co/k6a5xt﻿
Előfeldolgozás utáni komment:
['check', 'music', 'video', '!', 'fuego', 'u', 'la', 'la', 'remix', 'URL']


## Vektorizálás

In [6]:
# vektorizálás először count vektorizátorral
"""vectorizer = CountVectorizer(lowercase=False,
                             ngram_range=(1, 1), 
                             analyzer=preprocessor, 
                             max_df=0.95, 
                             min_df=5, 
                             max_features=None)
"""
# tfidf vektorizátor
vectorizer = TfidfVectorizer(lowercase=False,
                            ngram_range=(1,1),
                            analyzer=preprocessor,
                            max_df=0.95,
                            min_df=5)
X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['spam'].map(lambda x: int(x))
X_test = vectorizer.transform(test_df['text'])
y_test = test_df['spam'].map(lambda x: int(x))

## Feature engineering - extra features

Néhány spam komment kilistázása után feltűnt, hogy a spamok gyakran használnak nagybetűket, felkiáltójeleket. Hogy a tanítókorpuszban is megjelenjenek ezek a jellemzők 4 extra feature-t kiszámító funkciót írtam. Ezek a betűk arányát, a nagybetűk arányát, az írásjelek arányát és a posztolás idejét napszakra lebontva adják vissza. Az extra feature-öket a vektorizálás utáni mátrixokhoz fűztem hozzá.

In [7]:
def calc_char_ratios(comment):
    if len(comment) < 1:
        return (0, 0, 0)
    # betűk aránya
    lett_ratio = len(re.findall(r'[a-zA-Z]', comment))/len(comment)
    # nagybetűk aránya
    upper_ratio = len(re.findall(r'[A-Z]', comment))/len(comment)
    # irasjelek aránya
    punc_ratio = len(re.findall(r'[^a-zA-Z0-9\s]', comment))/len(comment)
    return (lett_ratio, upper_ratio, punc_ratio)
    
def deter_posting_time(date):
    # talán a komment posztolásának ideje a spamok esetében eltér a normál felhasználók posztolási idejétől,
    # ami főleg akkor lehet jelentős, ha a spam kommenteket gépek küldik
    # posztolás ideje: 0 = NaN, 1 = hajnal, 2 = delelott, 3 = delutan, 4 = este 
    if len(date) < 1:
        return 0
    hour = int(re.search(r'T(\d{2}):', date).groups()[0])
    if hour < 6:
        return 1
    elif hour < 12:
        return 2
    elif hour < 18:
        return 3
    else:
        return 4

In [8]:
# extra feature-ök hozzáadása a tanítóadatokhoz
n = X_train.shape[1]
feature_names = ['lett_ratio', 'upper_ratio', 'punc_ratio', 'posting_time']
extra_train = train_df['text'].map(lambda x: calc_char_ratios(x))
extra_train = extra_train.apply(pd.Series)
extra_train[3] = train_df['date'].map(lambda x: deter_posting_time(x))
X_train = hstack((X_train, extra_train))

#update vecotorizer vocabulary
for i in range(4):
    vectorizer.vocabulary_[feature_names[i]] = n + i

# extra feature-ök hozzáadása a tesztadatokhoz
extra_test = test_df['text'].map(lambda x: calc_char_ratios(x))
extra_test = extra_test.apply(pd.Series)
extra_test[3] = test_df['date'].map(lambda x: deter_posting_time(x))
X_test = hstack((X_test, extra_test))

print(X_train.shape, X_test.shape)

(1367, 401) (586, 401)


## Multinomial Naive Bayes classifier

A spamdetektálás egy klasszifikációs feladat, amelyet az irodolomban található eredmények szerint a Naív Bayes osztályozók kifejezetten jól képesek kezelni. A Naív Bayes-féle osztályozók továbbá kellően robosztusak, gyorsak és kevés tanulóadattal is jól működnek, így baseline modellként érdemes ezzel kezdeni.

Mivel a nyelvi adatok feldolgozásához gyakorisági alapú vektorizálót használtam, így a naív Bayes osztályozók multinomiális változatát választottam.

In [9]:
multi_nb = MultinomialNB()
multi_nb.fit(X_train, y_train)
print('Recall a tanulóadatokon')
print(classification_report(y_train, multi_nb.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, multi_nb.predict(X_test)))
# performance logs
# CountVectorizer max_df 1.0, min_df 1, MN Naive Bayes (alpha 1), test set: 0 recall 0.84, 1 recall 0.95
# CountVectorizer max_df 1.0, min_df 1, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.89, 1 recall 0.95
# CountVectorizer max_df 0.95, min_df 5, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.96, 1 recall 0.88
# TfidfVectorizer max_df 1.0, min_df 1, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.95, 1 recall 0.91
# TfidfVectorizer max_df 0.95, min_df 5, MN Naive Bayes (alpha 1) with extra features, test set: 0 recall 0.93, 1 recall 0.94

Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.93      0.97      0.95       665
          1       0.97      0.93      0.95       702

avg / total       0.95      0.95      0.95      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.90      0.93      0.92       285
          1       0.93      0.91      0.92       301

avg / total       0.92      0.92      0.92       586



In [10]:
# alpha állítása - Grid Search Cross Validation
mn_nb = MultinomialNB()
params = {'alpha': [0.01, 0.1, 1, 2, 5, 10]}
grid_CV = GridSearchCV(mn_nb, params, cv=10, refit=True)
grid_CV.fit(X_train, y_train)
print('Best alpha: {}'.format(grid_CV.best_params_))
print('Recall a tanulóadatokon')
print(classification_report(y_train, grid_CV.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, grid_CV.predict(X_test)))
# best model with TfidfVectorizer max_df 0.95, min_df 5, with extra featerus
# MN_Bayes alpha 2, 0 recall on test dataset 0.94; 1 recall on test dataset 0.94

Best alpha: {'alpha': 1}
Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.93      0.97      0.95       665
          1       0.97      0.93      0.95       702

avg / total       0.95      0.95      0.95      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.90      0.93      0.92       285
          1       0.93      0.91      0.92       301

avg / total       0.92      0.92      0.92       586



## SVM

A Support Vector Machine algoritmusok ugyancsak jól szoktak szerepelni a szövegkategorizációs feladatokban, így a következő részben megnézem, hogy milyen pontossággal képesek a kommentek közül kiválogatni a spamokat. A Naív Bayes-féle modellek eddigi legjobbjai 94%-os recall értékek körül teljesítettek.

In [11]:
svc_model = SVC()
svc_params = {'C': [0.01, 0.1, 1, 10],
             'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
             'degree': [2],
             'class_weight': ['balanced']}
svc_grid = GridSearchCV(svc_model, svc_params, cv=10, refit=True)
svc_grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.1, 1, 10], 'degree': [2], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [12]:
print('Best params: {}'.format(svc_grid.best_params_))
print('Recall a tanulóadatokon')
print(classification_report(y_train, svc_grid.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, svc_grid.predict(X_test)))

Best params: {'C': 1, 'degree': 2, 'kernel': 'linear', 'class_weight': 'balanced'}
Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.95      0.99      0.97       665
          1       0.99      0.95      0.97       702

avg / total       0.97      0.97      0.97      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.92      0.98      0.95       285
          1       0.98      0.92      0.95       301

avg / total       0.95      0.95      0.95       586



Az SVM klasszifikátor 1%-kal teljesít jobban, mint a Naív Bayes modell. Átlagosan a recall 95% a tesztadatokon. Mivel a legjobb eredményt a lineáris kernel érte el, így érdemes lehet logaritmikus regresszióval is ellenőrizni az eredményeket, hiszen a scikit-learn Logistic Regression moduljában be lehet állítani "L1" loss funkciót, míg az SVC-ben erre nincs lehetőség.

## Logistic Regression

In [13]:
logreg_model = LogisticRegression()
logreg_params = {'penalty': ['l1', 'l2'],
                'C': [0.01, 0.1, 1, 10],
                'class_weight': ['balanced']}
logreg_grid = GridSearchCV(logreg_model, logreg_params, cv=10, refit=True)
logreg_grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1, 10], 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [14]:
print('Best params: {}'.format(logreg_grid.best_params_))
print('Recall a tanulóadatokon')
print(classification_report(y_train, logreg_grid.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, logreg_grid.predict(X_test)))

Best params: {'C': 1, 'penalty': 'l2', 'class_weight': 'balanced'}
Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       0.93      0.98      0.95       665
          1       0.98      0.93      0.95       702

avg / total       0.96      0.95      0.95      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.91      0.97      0.94       285
          1       0.97      0.91      0.94       301

avg / total       0.94      0.94      0.94       586



Habár a logaritmikus regresszió az "L2" loss funkcióval érte el a legjobb eredményt, a lineáris SVM-hez képest a modell 1%-kal jobban teljesít: átlagosan 96%-os pontosságál kategorizál a modell a teszthalmazon.

## Random Forest

Az utolsó modellként a Random Forest algoritmus teljesítményét tesztelem, amely a kategorizációs feladatokban jellemzően jól szokott teljesíteni.
A Random Forest algoritmusok nagy hátránya, hogy sokáig tart tanítani őket. Mivel azonban a jelen adatbázis viszonylag kevés tanítóadatot tartalmaz, így remélhetőleg a random forest megbírkózik vele.

In [15]:
rf_model = RandomForestClassifier()
rf_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
rf_grid = GridSearchCV(rf_model, rf_grid, cv=10, refit=True)
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'bootstrap': [True, False], 'min_samples_leaf': [1, 3, 10], 'min_samples_split': [1, 3, 10], 'max_depth': [3, None], 'criterion': ['gini', 'entropy'], 'max_features': [1, 3, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [16]:
print('Best params: {}'.format(rf_grid.best_params_))
print('Recall a tanulóadatokon')
print(classification_report(y_train, rf_grid.predict(X_train)))
print('Recall a tesztadatokon')
print(classification_report(y_test, rf_grid.predict(X_test)))

Best params: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'criterion': 'entropy', 'min_samples_split': 10, 'max_features': 10}
Recall a tanulóadatokon
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       665
          1       1.00      1.00      1.00       702

avg / total       1.00      1.00      1.00      1367

Recall a tesztadatokon
             precision    recall  f1-score   support

          0       0.95      0.98      0.96       285
          1       0.98      0.95      0.96       301

avg / total       0.96      0.96      0.96       586



Eddig a Random Forest terjesített a legjobban, amely a tesztadatokon 96-97%-os recall pontossággal kategorizálta a spam és a nem spam kommenteket. Ez azt jelenti, hogy a 586 tesztpélda küzöl a Random Forest csak 20 kommentet kategorizált félre. Ezek az eredmények meglehetősen ígéretesek, így ha a minta reprezentatív a populációra nézve, a tanított modellek 95% fölötti pontossággal meg tudják állapítani egy adott kommentről, hogy spam vagy sem. Az osztályozók teljesítményét talán még 1-2%-kal lehetne javítani az előfeldozó és a extra feature-ök fejlesztésével.